In [ ]:
from skimage import io
import numpy as np
from libsvm.svmutil import *  
from libsvm.svm import *  
from sklearn import svm
from PIL import Image 
import time
import os
import sys
from sklearn import decomposition
from scipy.ndimage import binary_hit_or_miss

In [ ]:
TrainingSampleNum = 2000 # 学習サンプル総数
TestSampleNum = 10000 # テストサンプル総数
ClassNum = 10 # クラス数（今回は10）
ImageSize = 28 # 画像サイズ（今回は縦横ともに28）
#TrainingDataFile = './Images/TrainingSamples/{0:1d}-{1:04d}.png'
TestDataFile = './Images/TestSamples/{0:1d}-{1:04d}.png'
OutFile = './Images/OutSamples/thined_{0:1d}-{1:04d}.png' 
TrainingDataFile = './Images/OutSamples/bin_{0:1d}-{1:04d}.png'

In [ ]:
def Binarization ( src, thres ):
    '''
    ここでは画素の操作方法がわかるようにあえて２重ループで書いている．
    '''
    
    dest = np.zeros(src.shape, dtype=np.uint8)
    for y in range(0, src.shape[0]):
        for x in range(0, src.shape[1]):
            if src[y,x] < thres:
                dest[y,x] = 255
            else:
                dest[y,x] = 0

    return dest

In [ ]:
# Zhang-Suenのアルゴリズムを用いて2値化画像を細線化します
def Zhang_Suen_thinning(binary_image):
    # オリジナルの画像をコピー
    image_thinned = binary_image.copy()
    # 初期化します。この値は次のwhile文の中で除かれます。
    changing_1 = changing_2 = [1]
    while changing_1 or changing_2:
        # ステップ1
        changing_1 = []
        rows, columns = image_thinned.shape
        for x in range(1, rows - 1):
            for y in range(1, columns -1):
                p2, p3, p4, p5, p6, p7, p8, p9 = neighbour_points = neighbours(x, y, image_thinned)
                if (image_thinned[x][y] == 1 and
                    2 <= sum(neighbour_points) <= 6 and # 条件2
                    count_transition(neighbour_points) == 1 and # 条件3
                    p2 * p4 * p6 == 0 and # 条件4
                    p4 * p6 * p8 == 0): # 条件5
                    changing_1.append((x,y))
        for x, y in changing_1:
            image_thinned[x][y] = 0
        # ステップ2
        changing_2 = []
        for x in range(1, rows - 1):
            for y in range(1, columns -1):
                p2, p3, p4, p5, p6, p7, p8, p9 = neighbour_points = neighbours(x, y, image_thinned)
                if (image_thinned[x][y] == 1 and
                    2 <= sum(neighbour_points) <= 6 and # 条件2
                    count_transition(neighbour_points) == 1 and # 条件3
                    p2 * p4 * p8 == 0 and # 条件4
                    p2 * p6 * p8 == 0): # 条件5
                    changing_2.append((x,y))
        for x, y in changing_2:
            image_thinned[x][y] = 0        

    return image_thinned

# 2値画像の黒を1、白を0とするように変換するメソッドです
def black_one(binary):
    bool_image = binary.astype(bool)
    inv_bool_image = ~bool_image
    return inv_bool_image.astype(int)

# 画像の外周を0で埋めるメソッドです
def padding_zeros(image):
    import numpy as np
    m,n = np.shape(image)
    padded_image = np.zeros((m+2,n+2))
    padded_image[1:-1,1:-1] = image
    return padded_image

# 外周1行1列を除くメソッドです。
def unpadding(image):
    return image[1:-1, 1:-1]

# 指定されたピクセルの周囲のピクセルを取得するメソッドです
def neighbours(x, y, image):
    return [image[x-1][y], image[x-1][y+1], image[x][y+1], image[x+1][y+1], # 2, 3, 4, 5
             image[x+1][y], image[x+1][y-1], image[x][y-1], image[x-1][y-1]] # 6, 7, 8, 9

# 0→1の変化の回数を数えるメソッドです
def count_transition(neighbours):
    neighbours += neighbours[:1]
    return sum( (n1, n2) == (0, 1) for n1, n2 in zip(neighbours, neighbours[1:]) )

# 黒を1、白を0とする画像を、2値画像に戻すメソッドです
def inv_black_one(inv_bool_image):
    bool_image = ~inv_bool_image.astype(bool)
    return bool_image.astype(int) * 255

In [ ]:
def img2vector(imgFile, thres):
    img = Image.open(imgFile).convert('L')
    img_arr = np.array(img, 'i') # 20px * 20px 灰度图像
    img_normlization = np.zeros(img_arr.shape, dtype=np.uint8)
    for y in range(0, img_arr.shape[0]):
        for x in range(0, img_arr.shape[1]):
            if img_arr[y,x] < thres:
                img_normlization[y,x] = 1
            else:
                img_normlization[y,x] = 0
    #img_normlization = np.round(img_arr/255) # 对灰度值进行归一化
    #img_arr2 = np.reshape(img_normlization, (1,-1))  # 1 * 784 矩阵
    
    img_2 = padding_zeros(img_normlization)
    img_3 = Zhang_Suen_thinning(img_2)
    img_4 = inv_black_one(unpadding(img_3))
    return img_4

In [ ]:
def read_and_convert(imgFileList, label):
    dataLabel = [] # 存放类标签
    dataNum = len(imgFileList)
    dataMat = np.zeros((dataNum, 784)) # dataNum * 784 的矩阵
    for i in range(dataNum):
        imgName = imgFileList[i]
        #imgName = get_img_name_str(imgNameStr)  # 得到 数字_实例编号.png
        #print("imgName: {}".format(imgName))
        #sclassTag = imgName.split(".")[0].split("-")[0] # 得到 类标签(数字)
        #print("classTag: {}".format(classTag))
        dataLabel.append(label)
        dataMat[i,:] = img2vector(imgName, 30)
        
    return dataMat, dataLabel

In [ ]:
def read_all_data():
    flist = [ None ] * 200
    flist_ = [ None ] * 200
    for sample in range(0, TrainingSampleNum // ClassNum):
        flist[sample] = TrainingDataFile.format(0, sample)
    dataMat, dataLabel = read_and_convert(flist, 0)
    for label in range(1, ClassNum):
        for sample in range(0, TrainingSampleNum // ClassNum):
            flist_[sample] = TrainingDataFile.format(label, sample)
        dataMat_, dataLabel_ = read_and_convert(flist_, label)
        dataMat = np.concatenate((dataMat, dataMat_), axis=0)
        dataLabel = np.concatenate((dataLabel, dataLabel_), axis=0)
    return dataMat, dataLabel

In [ ]:
# main ルーチン

for label in range(0, ClassNum):
    for sample in range(0, TrainingSampleNum // ClassNum):
        filename = TrainingDataFile.format(label, sample)
        print ("Loading the file: " + filename)
        #img = io.imread(filename)
        
        res = img2vector(filename, 30)
            
        filename = OutFile.format(label, sample)
        print("Saving the file: " + filename)
        io.imsave(filename, res)